In [ ]:
# all new rna lines in 21Q3 had wrong internal bam (& sizehash cols) assigned on the tracker
# this notebook fixes 

from __future__ import print_function
import pandas as pd

import os 
from depmapomics import tracker

In [ ]:
# investigate why internal_bam and internal_bai are swapped within a set of samples
track = tracker.getTracker()
track = track.dropna(subset=['internal_bam_filepath', 'internal_bai_filepath'])
track['cds_id'] = track.index

In [ ]:
swappedbams = track[track.apply(lambda x: 'CDS-' in x.internal_bam_filepath and x.cds_id not in x.internal_bam_filepath, axis=1)]

In [ ]:
swappedbams

In [ ]:
swappedbams.cds_id.tolist()

In [ ]:
# making sure there's no bams missing/broken, just swapped within this set
# set([x.split('/')[-1][:10] for x in swappedbams['internal_bam_filepath']]) - set(swappedbams.index)
set(swappedbams.index) == set([x.split('/')[-1][:10] for x in swappedbams['internal_bam_filepath']])

In [ ]:
set(swappedbams.index) == set([x.split('/')[-1][:10] for x in swappedbams['internal_bai_filepath']])